In [1]:
import pandas as pd

In [2]:
moves = pd.read_csv("/home/kelleyld/OTBa_moves.csv")

In [3]:
moves.head()

,GameID,Turn,Ply,MovedPiece,SourceCol,SourceRow,DestCol,DestRow,CapturedPiece,PromotionPiece,CheckState,CastleSide
0,1,w,1,P,4,1,4,3,NaN,NaN,NaN,NaN
1,1,b,2,p,3,6,3,4,NaN,NaN,NaN,NaN
2,1,w,3,P,4,3,3,4,p,NaN,NaN,NaN
3,1,b,4,q,3,7,3,4,P,NaN,NaN,NaN
4,1,w,5,N,1,0,2,2,NaN,NaN,NaN,NaN


In [11]:
def convert_data_frame(moves):
    print(moves.info())
    print()
    
    categories = ["Turn", "MovedPiece", "SourceCol", "SourceRow", "DestCol", "DestRow",
              "CapturedPiece", "PromotionPiece", "CheckState", "CastleSide"]
    col_map = {
        0 : "a", 
        1 : "b",
        2 : "c",
        3 : "d",
        4 : "e",
        5 : "f",
        6 : "g",
        7 : "h"
    }
    numerics = ["GameID", "Ply"]
    
    def get_row(i):
        return i + 1
    
    moves["SourceCol"] = moves.loc[:, "SourceCol"].map(col_map)
    moves["DestCol"] = moves.loc[:, "DestCol"].map(col_map)
    moves["SourceRow"] = moves.loc[:, "SourceRow"].map(get_row)
    moves["DestRow"] = moves.loc[:, "DestRow"].map(get_row)
    
    for category in categories:
        print("space before:", moves.loc[:, category].nbytes)
        moves[category] = moves.loc[:, category].astype("category")
        print("      after: ", moves.loc[:, category].nbytes)
    
    for numeric in numerics:
        print("space before: ", moves.loc[:, numeric].nbytes)
        moves[numeric] = pd.to_numeric(moves.loc[:, numeric])
        print("      after:  ", moves.loc[:, numeric].nbytes)
    
    print()
    print(moves.info())    
    return moves

In [4]:
moves = convert_data_frame(moves)

space before: 547960760
      after:  68495111
space before: 547960760
      after:  68495191
space before: 547960760
      after:  68495159
space before: 547960760
      after:  68495159
space before: 547960760
      after:  68495159
space before: 547960760
      after:  68495159
space before: 547960760
      after:  68495175
space before: 547960760
      after:  68495127
space before: 547960760
      after:  68495111
space before: 547960760
      after:  68495111
space before:  547960760
      after:   547960760
space before:  547960760
      after:   547960760
GameID               int64
Turn              category
Ply                  int64
MovedPiece        category
SourceCol         category
SourceRow         category
DestCol           category
DestRow           category
CapturedPiece     category
PromotionPiece    category
CheckState        category
CastleSide        category
dtype: object


In [7]:
moves.to_hdf("./OTBa_moves.h5", "chess_move", mode="w", format="table")

In [12]:
moves = pd.read_hdf("./OTBa_moves.h5")
moves.head()

,GameID,Turn,Ply,MovedPiece,SourceCol,SourceRow,DestCol,DestRow,CapturedPiece,PromotionPiece,CheckState,CastleSide
0,1,w,1,P,e,2,e,4,NaN,NaN,NaN,NaN
1,1,b,2,p,d,7,d,5,NaN,NaN,NaN,NaN
2,1,w,3,P,e,4,d,5,p,NaN,NaN,NaN
3,1,b,4,q,d,8,d,5,P,NaN,NaN,NaN
4,1,w,5,N,b,1,c,3,NaN,NaN,NaN,NaN


In [31]:
piece_used_to_capture = moves[moves["CapturedPiece"].notnull()].groupby(["MovedPiece", "CapturedPiece"]).count().dropna()

In [32]:
piece_used_to_capture.head()

GameID      Turn       Ply  SourceCol  SourceRow  \
MovedPiece CapturedPiece                                                       
B          b              342145.0  342145.0  342145.0   342145.0   342145.0   
           n              430070.0  430070.0  430070.0   430070.0   430070.0   
           p              376438.0  376438.0  376438.0   376438.0   376438.0   
           q               45117.0   45117.0   45117.0    45117.0    45117.0   
           r               95227.0   95227.0   95227.0    95227.0    95227.0   

                           DestCol   DestRow  PromotionPiece  CheckState  \
MovedPiece CapturedPiece                                                   
B          b              342145.0  342145.0             0.0     28881.0   
           n              430070.0  430070.0             0.0     19432.0   
           p              376438.0  376438.0             0.0     38215.0   
           q               45117.0   45117.0             0.0      2148.0   
           r               95227.0   95227.0             0.0      4261.0   

                          CastleSide  
MovedPiece CapturedPiece              
B          b                     0.0  
           n                     0.0  
           p                     0.0  
           q                     0.0  
           r                     0.0

In [41]:
position_vulnerability = moves[moves["CapturedPiece"].notnull()].groupby(["DestCol", "DestRow", "SourceCol", "SourceRow"]).count().dropna()
position_vulnerability["attacked_from_source"] = position_vulnerability["GameID"]
position_vulnerability["attacked_from_source"]

DestCol  DestRow  SourceCol  SourceRow
a        1        a          2            1122.0
                             3             565.0
                             4             636.0
                             5             888.0
                             6             585.0
                             7             722.0
                             8            8433.0
                  b          1            1712.0
                             2            4460.0
                             3            1719.0
                  c          1            1714.0
                             2            2741.0
                             3            2215.0
                  d          1            4765.0
                             4            1579.0
                  e          1            2537.0
                             5             969.0
                  f          1            4275.0
                             6            1288.0
                  g          1

In [43]:
pd.DataFrame(position_vulnerability["attacked_from_source"]).groupby(["DestCol", "DestRow"])["attacked_from_source"].sum()

DestCol  DestRow
a        1            46674.0
         2            80761.0
         3            86928.0
         4           132785.0
         5           119715.0
         6           118657.0
         7            79621.0
         8            58145.0
b        1            34048.0
         2           135807.0
         3           185720.0
         4           279419.0
         5           331933.0
         6           171281.0
         7           135535.0
         8            38773.0
c        1            91529.0
         2            97225.0
         3           460181.0
         4           480516.0
         5           458133.0
         6           417917.0
         7            96637.0
         8           105061.0
d        1           137699.0
         2           168838.0
         3           235672.0
         4          1134552.0
         5          1069723.0
         6           305127.0
                      ...    
e        3           229184.0
         4           70

In [45]:
moves.GameID.unique().max()

933092